In [1]:
%config PPMagics.domain="ccg24-hrzana-fpa-global-bi"
%reload_ext cloudmagics.bigquery

### Rcvr Table

In [2]:
create or replace table `pypl-edw`.pp_oap_mugilamanivannan_t.merchant_base_table
as 
(
select 
a.mth_id,
parse_date('%Y-%m',b.mth_yyyy_mm_name) as transaction_month,
a.cust_id,
a.pp_bt_cust_id,
a.cust_acct_clsfn_key,
a.sales_seg,
a.market,
a.sub_market,
CASE WHEN sub_market in ('UK', 'CHINA') then sub_market
when cntry_code in ('US', 'CA', 'DE', 'AU', 'FR', 'IT','ES', 'BR','MX', 'JP') then cntry_code
when sub_market in ('DEATCH', 'ICE', 'NEU', 'SEU') and cntry_code not in ('US', 'CA', 'DE', 'AU', 'FR', 'IT','ES', 'BR','MX', 'JP', 'CN') then 'Rest of Europe'
else 'Rest of World' end as Region,  
a.cntry_code,   
CASE WHEN upper(trim(a.prod_hier_desc))='BRAINTREE' then 'Braintree' 
     WHEN upper(trim(a.prod_hier_l1)) IN ('BRANDED WALLET-DIRECT', 'BRANDED WALLET-MARKETPLACE','BRANDED WALLET-PARTNERS', 'EBAY') THEN 'Wallet'
     WHEN upper(trim(a.prod_hier_l1)) IN ('UNBRANDED CARD PROCESSING-DIRECT', 'UNBRANDED CARD PROCESSING-MARKETPLACE','UNBRANDED CARD PROCESSING-PARTNERS') THEN 'PP-Cards'     
     WHEN upper(trim(a.prod_hier_l1)) IN ('APM-DIRECT', 'APM-MARKETPLACE', 'APM-PARTNERS', 'PWV-DIRECT', 'PWV-MARKETPLACE', 'PWV-PARTNERS',
                                         'CREDIT-DIRECT', 'CREDIT-MARKETPLACE', 'CREDIT-PARTNERS') THEN 'Other-XO'
     ELSE 'Non-XO'
END prod_grp_l1,   
CASE WHEN upper(trim(a.prod_hier_desc))='BRAINTREE' then 'Braintree' -- Braintree 
     WHEN upper(trim(a.prod_hier_l3)) in ('EC - LATEST', 'EC - BETTER', 'EC - LEGACY', 'PUI - BETTER', 'PUI - LEGACY') and upper(trim(a.prod_hier_desc)) <> 'WPS NON RECURRING' then 'EC' --Wallet
     WHEN (upper(trim(a.prod_hier_l3)) in ('WPS - LEGACY') or upper(trim(a.prod_hier_desc)) = 'WPS NON RECURRING') and (upper(trim(a.prod_hier_l3)) <> 'INTEGRATED QR') then 'WPS'   --Wallet
     WHEN upper(trim(a.prod_hier_l1)) = 'EBAY' THEN 'eBay'  --Wallet
     WHEN upper(trim(a.prod_hier_l2)) in ('ACDC', 'BCDC', 'PRO DCC') THEN a.prod_hier_l2  -- PP Cards
     WHEN upper(trim(a.prod_hier_l1)) like 'UNBRANDED CARD PROCESSING' and upper(trim(a.prod_hier_l2)) not in ('ACDC', 'BCDC', 'PRO DCC', 'BRAINTREE') THEN 'PPCards-Oth' -- PP Cards
     WHEN upper(trim(a.prod_hier_l1)) IN ('APM-DIRECT', 'APM-MARKETPLACE', 'APM-PARTNERS','PWV-DIRECT', 'PWV-MARKETPLACE',
                                          'PWV-PARTNERS') THEN 'APM (INCL PWV)' -- Other XO
     WHEN upper(trim(a.prod_hier_l1)) IN ('CREDIT-DIRECT', 'CREDIT-MARKETPLACE', 'CREDIT-PARTNERS') THEN 'Credit' -- Other XO
     WHEN upper(trim(a.prod_hier_l2)) in ('INVOICING', 'ZETTLE', 'PAYOUTS') then a.prod_hier_l2
     WHEN upper(trim(a.prod_hier_l2)) in ('P2P-F&F', 'P2P-G&S') then 'P2P'
     ELSE 'Oth'
END AS prod_grp_l2,
CASE WHEN upper(trim(a.prod_hier_desc)) = 'BRAINTREE' then 'Braintree' -- Braintree
     WHEN upper(trim(a.prod_hier_l1)) = 'EBAY' THEN 'eBay' --Wallet
     WHEN upper(trim(a.prod_hier_desc)) = 'MS EC RECURRING' and upper(trim(a.prod_hier_l2)) not in ('APM' , 'PWV', 'PPC 2ND BTN', 'ACDC', 'BCDC') then 'EC Recurring'  --Wallet
     WHEN upper(trim(a.prod_hier_desc)) = 'MS EC Non Recurring' and upper(trim(a.prod_hier_l2)) not in ('APM' , 'PWV', 'PPC 2ND BTN', 'ACDC', 'BCDC') then 'EC Non Recurring' --Wallet
     WHEN upper(trim(a.prod_hier_desc)) in ('WPS RECURRING', 'WPS NON RECURRING')  and  upper(trim(a.prod_hier_l2)) not in ('APM' , 'PWV', 'PPC 2ND BTN', 'ACDC', 'BCDC', 'INTEGRATED QR')  then a.prod_hier_desc --Wallet
     WHEN upper(trim(a.prod_hier_l3)) in ('EC - LATEST', 'EC - BETTER', 'EC - LEGACY', 'PUI - BETTER', 'PUI - LEGACY') then 'EC Non Recurring' --Wallet
     WHEN upper(trim(a.prod_hier_l3)) in ('WPS - LEGACY') then 'WPS Non Recurring'  --Wallet
     WHEN upper(trim(a.prod_hier_l2)) in ('ACDC', 'BCDC', 'PRO DCC') THEN a.prod_hier_l2 -- PP Cards     
     WHEN upper(trim(a.prod_hier_l2)) = 'HSS UNBRANDED' then 'HSS' -- PP Cards 
     WHEN upper(trim(a.prod_hier_l2)) = 'PP PLUS - UNBRANDED' then 'PP Plus'  -- PP Cards 
     WHEN upper(trim(a.prod_hier_l2)) = 'PPFM UNBRANDED' then 'PPFM'  -- PP Cards 
     WHEN upper(trim(a.prod_hier_l1)) IN ('APM-DIRECT', 'APM-MARKETPLACE', 'APM-PARTNERS') THEN 'APM'  -- Other XO
     WHEN upper(trim(a.prod_hier_l1)) IN ('PWV-DIRECT', 'PWV-MARKETPLACE', 'PWV-PARTNERS') THEN 'PWV'  -- Other XO
     WHEN upper(trim(a.prod_hier_l1)) IN ('CREDIT-DIRECT', 'CREDIT-MARKETPLACE', 'CREDIT-PARTNERS') THEN 'Credit'  -- Other XO
     WHEN upper(trim(a.prod_hier_l2)) in ('INVOICING', 'ZETTLE', 'P2P-G&S', 'VIRTUAL TERMINAL', 'PAYPAL HERE') then 
         upper(trim(a.prod_hier_l2))
     WHEN upper(trim(a.prod_hier_l1)) IN ('P2P-F&F RECV', 'P2P-F&F SEND', 'MS OTHERS', 'PAYOUTS RECV', 'PAYOUTS SEND') 
         then upper(trim(a.prod_hier_l1))
     WHEN upper(trim(a.prod_hier_l2)) in ('INTEGRATED QR', 'STATIC QR') then 'QRC' 
     WHEN upper(trim(a.prod_hier_l2)) = 'TOKENIZATION & DISCOVER POS' THEN 'POS'
     ELSE 'MS OTHERS'
END AS prod_grp_l3,
case when a.trade_flow = 'NA' then 'BT'
else a.trade_flow
end as trade_flow,
a.txn_prod_channel,
a.new_x_sell_retention_flag,
-- a.eds_flag,
-- a.ews_flag,
-- a.ems_flag,
case 
when new_flag='Y' then 'New'
when eds_flag='Y' then 'EDS'
when ews_flag='Y' then 'EWS'
when ems_flag='Y' then 'EMS'
else 'Rest' end as Activity_seg,   
mercury_sector_name as mercury_sector_name,
mercury_industry_name as mercury_industry_name,
CASE 
WHEN upper(trim(m.cust_id)) is not null or upper(trim(a.prod_hier_desc))='EBAY MOR' then 'MANAGED PATH' 
WHEN upper(trim(a.txn_prod_channel)) in ('PARTNER','MARKETPLACE') or 
upper(trim(a.prod_hier_desc))='EBAY NON-MOR' then 'CONNECTED PATH'
else 'DIRECT' 
END as Managed_flag,
CASE
WHEN a.cust_acct_type_code = 0 THEN 'Personal'
WHEN a.cust_acct_type_code = 1 THEN 'Premier'
ELSE 'Business'
END AS account_type,
sum(rcv_txn_cnt)  txn_cnt,
sum(a.rcv_ntpv_pln_usd_amt ) ntpv,
sum(a.net_rev_pln_usd_amt) AS revenue,
sum(a.net_margin_pln_usd_amt) AS margin,
sum(a.net_cost_pln_usd_amt) AS cost,
sum(a.net_loss_pln_usd_amt) AS loss
from pypl-edw.pp_ent_anlytcs_views.fact_merch_prod_hier_mth a
--LEFT JOIN `pypl-edw`.pp_discovery_views.dim_customer AS attr 
--ON upper(rtrim(a.pp_bt_cust_id)) = upper(rtrim(attr.cust_id))
LEFT OUTER JOIN `pypl-edw`.pp_discovery_views.dim_calendar_month AS b 
ON trim(CAST(a.mth_id as STRING)) = trim(CAST(b.mth_id as STRING))
LEFT JOIN (select cust_id,mercury_sector_name,mercury_industry_name from `pypl-edw`.pp_scratch.mercury_industry_details
where dt=(select max(dt) from pypl-edw.pp_scratch.mercury_industry_details)
group by 1,2,3) d on a.cust_id = d.cust_id  
LEFT JOIN  pypl-edw.pp_peak_views.dim_mam_merch_ssot m 
on upper(rtrim(a.pp_bt_cust_id)) = upper(rtrim(m.cust_id))
where a.mth_id>1452 and
    upper(trim(a.sales_seg)) in ('A1','A2','B1','B2','C','D','E','GLOBAL ACCOUNTS') and
    a.cust_acct_clsfn_key in (2,8,-99,21)
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21);

Establishing new BigQuery Client


HTML(value='')

job id =  g3pnDZgCwdKO7m6yU5p5IorHqZG
Time executed =  -63840675407.914  seconds


HTML(value='')

DownloadWidget(children=(HBox(children=(Dropdown(description='File Type:', index=1, options=('tde', 'csv', 'js…

mth_id,transaction_month,cust_id,pp_bt_cust_id,cust_acct_clsfn_key,sales_seg,market,sub_market,Region,cntry_code,prod_grp_l1,prod_grp_l2,prod_grp_l3,trade_flow,txn_prod_channel,new_x_sell_retention_flag,Activity_seg,mercury_sector_name,mercury_industry_name,Managed_flag,account_type,txn_cnt,ntpv,revenue,margin,cost,loss


### Sender Table

In [19]:
Insert into `pypl-edw`.pp_scratch.Send_Only_Merchants_Details
(
SELECT 
    'Non-Revenue generating' as cust_flag,
    a.mth_id,   
    parse_date('%Y-%m',d.mth_yyyy_mm_name) as transaction_month,
    a.cust_id,
    a.cust_id as pp_bt_cust_id, 
    cast(null as string) as entity_id,
    a.cust_acct_clsfn_key, 
    e.sales_seg, 
    b.fpa_rev_lvl_4 as market,    
    b.fpa_rev_lvl_3 as sub_market, 
    CASE WHEN b.fpa_rev_lvl_3 in ('UK', 'CHINA') then b.fpa_rev_lvl_3
    when a.cntry_code in ('US', 'CA', 'DE', 'AU', 'FR', 'IT','ES', 'BR','MX', 'JP') then a.cntry_code
    when b.fpa_rev_lvl_3 in ('DEATCH', 'ICE', 'NEU', 'SEU') and a.cntry_code not in ('US', 'CA', 'DE', 'AU', 'FR', 'IT','ES', 'BR','MX', 'JP', 'CN') then 'Rest of Europe'
    else 'Rest of World' end as Region,   
    a.cntry_code,   
    'Non_Rev_Send_Only' as prod_hier_desc,   
    'Non_Rev_Send_Only' as prod_hier_l1,      
    'Non_Rev_Send_Only' as prod_hier_l2,       
    'Non_Rev_Send_Only' as trade_flow,        
    'Non_Rev_Send_Only' as txn_prod_channel,   
    'Non_Rev_Send_Only' as new_x_sell_retention_flag,  
    'Non_Rev_Send_Only' as Activity_seg,  
    f.mercury_sector_name as mercury_sector_name,  
    f.mercury_industry_name as mercury_industry_name,  
    'Non_Rev_Send_Only' as Managed_flag, 
    CASE
    WHEN a.cust_acct_type_code = 0 THEN 'Personal'
    WHEN a.cust_acct_type_code = 1 THEN 'Premier'
    ELSE 'Business'
    END AS account_type,  
    0 as txn_cnt,
    0 as ntpv,
    0 as revenue, 
    0 as margin, 
    0 as cost,
    0 as loss 


FROM (SELECT * FROM `pypl-edw`.pp_discovery_views.fact_cust_prfl_mth_snap a
       WHERE mth_id = 1488 and cust_acct_clsfn_key in (2,8) and a.sndr_actv_seg_key in (1,2,3)
    and a.sndr_rcvr_actv_seg_key in (1,2,3) and a.rcvr_actv_seg_key not in (1,2,3)
     ) a
--JOIN (SELECT b.mth_id as cal_mth_id, a.*  
--        FROM pypl-edw.pp_ent_anlytcs_views.fact_user_actv_mth_snap a
--        JOIN  `pypl-edw`.pp_discovery_views.dim_calendar_month b
--          ON substring(cast(a.mth_id as string),1,7) = b.mth_yyyy_mm_name 
--       WHERE b.mth_id = 1486 and a.mth_clsfn_key in (2,8)) g   # This is used to extract the merchants who made a txn that particular month
-- ON a.cust_id = g.cust_id and a.mth_id = g.cal_mth_id  #10689059
JOIN `pypl-edw`.pp_discovery_views.dim_cntry b
ON a.cntry_code = b.cntry_code 
LEFT JOIN pypl-edw.pp_ent_anlytcs_views.fact_merch_prod_hier_mth c
ON a.cust_id = c.cust_id and a.mth_id = c.mth_id 
LEFT JOIN `pypl-edw`.pp_discovery_views.dim_calendar_month AS d
ON trim(CAST(a.mth_id as STRING)) = trim(CAST(d.mth_id as STRING))
LEFT JOIN `pypl-edw`.pp_ent_anlytcs_views.dim_busn_type_merch_snap e   -- left join with bus_snap table gives merchants who do not have a sales_seg but have a made a send activity
ON a.cust_id = e.cust_id and a.mth_id = e.mth_id 
LEFT JOIN (
select cust_id,mercury_sector_name,mercury_industry_name from pypl-edw.pp_scratch.mercury_industry_details
where dt=(select max(dt) from pypl-edw.pp_scratch.mercury_industry_details)
group by 1,2,3
) f
ON a.cust_id = f.cust_id 

WHERE c.cust_id is null
and a.mth_id = 1488
and a.cust_acct_clsfn_key in (2,8) 
and a.sndr_rcvr_actv_seg_key in (1,2,3)
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23);

HTML(value='')

job id =  CJeolOoJxawaWixexD8vmyJRBEu
Time executed =  -63840411848.893  seconds


HTML(value='')

DownloadWidget(children=(HBox(children=(Dropdown(description='File Type:', index=1, options=('tde', 'csv', 'js…

cust_flag,mth_id,transaction_month,cust_id,pp_bt_cust_id,entity_id,cust_acct_clsfn_key,sales_seg,market,sub_market,Region,cntry_code,prod_hier_desc,prod_hier_l1,prod_hier_l2,trade_flow,txn_prod_channel,new_x_sell_retention_flag,Activity_seg,mercury_sector_name,mercury_industry_name,Managed_flag,account_type,txn_cnt,ntpv,revenue,margin,cost,loss


#### Final Merchant Table  - Rcvr and Sender

In [3]:
create or replace table `pypl-bods`.prd_globalbi_sales_t.merchant_base_final_table_v2
as 
(
select 
'Revenue Generating' as flag,
a.mth_id,
a.transaction_month,
a.cust_id,
a.pp_bt_cust_id,
a.cust_acct_clsfn_key,
case when upper(trim(a.sales_seg)) in ('GLOBAL ACCOUNTS') then 'GA'
 When upper(trim(a.sales_seg)) in ('A1','A2','B1') then 'ES'
 When upper(trim(a.sales_seg)) in('B2') then 'MM'
 When upper(trim(a.sales_seg)) in ('C','D','E') then 'SB'
 Else '#'
 END as sales_seg,
a.market,
a.sub_market,
a.Region,  
a.cntry_code,   
a.prod_grp_l1,    
a.prod_grp_l2,
a.prod_grp_l3,
a.trade_flow,
a.txn_prod_channel,
a.new_x_sell_retention_flag,
a.Activity_seg,   
a.mercury_sector_name,
a.mercury_industry_name, 
a.Managed_flag,
a.account_type,
sum(a.txn_cnt) as txn_cnt,
sum(a.ntpv) as ntpv,
sum(a.revenue) as revenue,
sum(a.margin) as margin,
sum(a.cost) as cost,
sum(a.loss) as loss
from `pypl-edw`.pp_oap_mugilamanivannan_t.merchant_base_table a
    where mth_id >= 1475
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
union all
select
'All' as flag,
b.mth_id,
b.transaction_month,
b.cust_id,
b.pp_bt_cust_id,
b.cust_acct_clsfn_key,
b.sales_seg,
b.market,
b.sub_market,
b.Region,  
b.cntry_code,   
b.prod_grp_l1,    
b.prod_grp_l2,
b.prod_grp_l3,
b.trade_flow,
b.txn_prod_channel,
b.new_x_sell_retention_flag,
b.Activity_seg,   
b.mercury_sector_name,
b.mercury_industry_name, 
b.Managed_flag,
b.account_type,
sum(b.txn_cnt) as txn_cnt,
sum(b.ntpv) as ntpv,
sum(b.revenue) as revenue,
sum(b.margin) as margin,
sum(b.cost) as cost,
sum(b.loss) as loss
from 
    (
select 
a.mth_id,
a.transaction_month,
a.cust_id,
a.pp_bt_cust_id,
a.cust_acct_clsfn_key,
case when upper(trim(a.sales_seg)) in ('GLOBAL ACCOUNTS') then 'GA'
 When upper(trim(a.sales_seg)) in ('A1','A2','B1') then 'ES'
 When upper(trim(a.sales_seg)) in('B2') then 'MM'
 When upper(trim(a.sales_seg)) in ('C','D','E') then 'SB'
 Else '#'
 END as sales_seg,
a.market,
a.sub_market,
a.Region,  
a.cntry_code,   
a.prod_grp_l1,    
a.prod_grp_l2,
a.prod_grp_l3,
a.trade_flow,
a.txn_prod_channel,
a.new_x_sell_retention_flag,
a.Activity_seg,   
a.mercury_sector_name,
a.mercury_industry_name, 
a.Managed_flag,
a.account_type,
sum(a.txn_cnt) as txn_cnt,
sum(a.ntpv) as ntpv,
sum(a.revenue) as revenue,
sum(a.margin) as margin,
sum(a.cost) as cost,
sum(a.loss) as loss
from `pypl-edw`.pp_oap_mugilamanivannan_t.merchant_base_table a
    where mth_id >= 1475
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
union all
select
b.mth_id,
b.transaction_month,
b.cust_id,
b.pp_bt_cust_id,
b.cust_acct_clsfn_key,
case when upper(trim(b.sales_seg)) in ('GLOBAL ACCOUNTS') then 'GA'
 When upper(trim(b.sales_seg)) in ('A1','A2','B1') then 'ES'
 When upper(trim(b.sales_seg)) in('B2') then 'MM'
 When upper(trim(b.sales_seg)) in ('C','D','E') then 'SB'
 Else '#'
 END as sales_seg,
b.market,
b.sub_market,
b.Region,  
b.cntry_code,   
b.prod_hier_desc as prod_grp_l1,    
b.prod_hier_l1 as prod_grp_l2,
b.prod_hier_l2 as prod_grp_l3,    
case when b.trade_flow = 'send_only' then "Non_Rev_Send_Only"
        else b.trade_flow
        end as trade_flow,
b.txn_prod_channel,
b.new_x_sell_retention_flag,
b.Activity_seg,   
b.mercury_sector_name,
b.mercury_industry_name, 
b.Managed_flag,
b.account_type,
sum(b.txn_cnt) as txn_cnt,
sum(b.ntpv) as ntpv,
sum(b.revenue) as revenue,
sum(b.margin) as margin,
sum(b.cost) as cost,
sum(b.loss) as loss
from `pypl-edw`.pp_scratch.Send_Only_Merchants_Details b
    where mth_id >= 1475
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21)
    b
    where mth_id >= 1475
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22);

HTML(value='')

job id =  myBTyafGmovndS7Yg6F8YSSGyPe
Time executed =  -63840676065.035  seconds


HTML(value='')

DownloadWidget(children=(HBox(children=(Dropdown(description='File Type:', index=1, options=('tde', 'csv', 'js…

flag,mth_id,transaction_month,cust_id,pp_bt_cust_id,cust_acct_clsfn_key,sales_seg,market,sub_market,Region,cntry_code,prod_grp_l1,prod_grp_l2,prod_grp_l3,trade_flow,txn_prod_channel,new_x_sell_retention_flag,Activity_seg,mercury_sector_name,mercury_industry_name,Managed_flag,account_type,txn_cnt,ntpv,revenue,margin,cost,loss


In [5]:
Insert into `pypl-bods`.prd_globalbi_sales_t.merchant_base_final_table_v2
(
select 
'Revenue Generating' as flag,
a.mth_id,
a.transaction_month,
a.cust_id,
a.pp_bt_cust_id,
a.cust_acct_clsfn_key,
case when upper(trim(a.sales_seg)) in ('GLOBAL ACCOUNTS') then 'GA'
 When upper(trim(a.sales_seg)) in ('A1','A2','B1') then 'ES'
 When upper(trim(a.sales_seg)) in('B2') then 'MM'
 When upper(trim(a.sales_seg)) in ('C','D','E') then 'SB'
 Else '#'
 END as sales_seg,
a.market,
a.sub_market,
a.Region,  
a.cntry_code,   
a.prod_grp_l1,    
a.prod_grp_l2,
a.prod_grp_l3,
a.trade_flow,
a.txn_prod_channel,
a.new_x_sell_retention_flag,
a.Activity_seg,   
a.mercury_sector_name,
a.mercury_industry_name, 
a.Managed_flag,
a.account_type,
sum(a.txn_cnt) as txn_cnt,
sum(a.ntpv) as ntpv,
sum(a.revenue) as revenue,
sum(a.margin) as margin,
sum(a.cost) as cost,
sum(a.loss) as loss
from `pypl-edw`.pp_oap_mugilamanivannan_t.merchant_base_table a
    where mth_id between 1466 and 1474
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
union all
select
'All' as flag,
b.mth_id,
b.transaction_month,
b.cust_id,
b.pp_bt_cust_id,
b.cust_acct_clsfn_key,
b.sales_seg,
b.market,
b.sub_market,
b.Region,  
b.cntry_code,   
b.prod_grp_l1,    
b.prod_grp_l2,
b.prod_grp_l3,
b.trade_flow,
b.txn_prod_channel,
b.new_x_sell_retention_flag,
b.Activity_seg,   
b.mercury_sector_name,
b.mercury_industry_name, 
b.Managed_flag,
b.account_type,
sum(b.txn_cnt) as txn_cnt,
sum(b.ntpv) as ntpv,
sum(b.revenue) as revenue,
sum(b.margin) as margin,
sum(b.cost) as cost,
sum(b.loss) as loss
from 
    (
select 
a.mth_id,
a.transaction_month,
a.cust_id,
a.pp_bt_cust_id,
a.cust_acct_clsfn_key,
case when upper(trim(a.sales_seg)) in ('GLOBAL ACCOUNTS') then 'GA'
 When upper(trim(a.sales_seg)) in ('A1','A2','B1') then 'ES'
 When upper(trim(a.sales_seg)) in('B2') then 'MM'
 When upper(trim(a.sales_seg)) in ('C','D','E') then 'SB'
 Else '#'
 END as sales_seg,
a.market,
a.sub_market,
a.Region,  
a.cntry_code,   
a.prod_grp_l1,    
a.prod_grp_l2,
a.prod_grp_l3,
a.trade_flow,
a.txn_prod_channel,
a.new_x_sell_retention_flag,
a.Activity_seg,   
a.mercury_sector_name,
a.mercury_industry_name, 
a.Managed_flag,
a.account_type,
sum(a.txn_cnt) as txn_cnt,
sum(a.ntpv) as ntpv,
sum(a.revenue) as revenue,
sum(a.margin) as margin,
sum(a.cost) as cost,
sum(a.loss) as loss
from `pypl-edw`.pp_oap_mugilamanivannan_t.merchant_base_table a
    where mth_id between 1466 and 1474
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
union all
select
b.mth_id,
b.transaction_month,
b.cust_id,
b.pp_bt_cust_id,
b.cust_acct_clsfn_key,
case when upper(trim(b.sales_seg)) in ('GLOBAL ACCOUNTS') then 'GA'
 When upper(trim(b.sales_seg)) in ('A1','A2','B1') then 'ES'
 When upper(trim(b.sales_seg)) in('B2') then 'MM'
 When upper(trim(b.sales_seg)) in ('C','D','E') then 'SB'
 Else '#'
 END as sales_seg,
b.market,
b.sub_market,
b.Region,  
b.cntry_code,   
b.prod_hier_desc as prod_grp_l1, 
b.prod_hier_l1 as prod_grp_l2,
b.prod_hier_l2 as prod_grp_l3,
case when b.trade_flow = 'send_only' then "Non_Rev_Send_Only"
        else b.trade_flow
        end as trade_flow,
b.txn_prod_channel,
b.new_x_sell_retention_flag,
b.Activity_seg,   
b.mercury_sector_name,
b.mercury_industry_name, 
b.Managed_flag,
b.account_type,
sum(b.txn_cnt) as txn_cnt,
sum(b.ntpv) as ntpv,
sum(b.revenue) as revenue,
sum(b.margin) as margin,
sum(b.cost) as cost,
sum(b.loss) as loss
from `pypl-edw`.pp_scratch.Send_Only_Merchants_Details b
    where mth_id between 1466 and 1474
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21)
    b
    where mth_id between 1466 and 1474
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22);

HTML(value='')

job id =  4VQlBcXcDc53lSsevA3Y2nlfA9e
Time executed =  -63840676507.573  seconds


HTML(value='')

DownloadWidget(children=(HBox(children=(Dropdown(description='File Type:', index=1, options=('tde', 'csv', 'js…

flag,mth_id,transaction_month,cust_id,pp_bt_cust_id,cust_acct_clsfn_key,sales_seg,market,sub_market,Region,cntry_code,prod_grp_l1,prod_grp_l2,prod_grp_l3,trade_flow,txn_prod_channel,new_x_sell_retention_flag,Activity_seg,mercury_sector_name,mercury_industry_name,Managed_flag,account_type,txn_cnt,ntpv,revenue,margin,cost,loss


### Engagement KPI

In [6]:
%%ppbq
CREATE or replace TABLE `pypl-bods`.prd_globalbi_sales_t.merchant_base_engagement_cohort as
 
WITH cohort_data AS (
  SELECT
    cust_id,
    EXTRACT(YEAR FROM transaction_month) AS cohort_year,
  FROM
    `pypl-bods`.prd_globalbi_sales_t.merchant_base_final_table_v2
  GROUP BY
   1,2 
)
 
SELECT 
  b.cohort_year,
  EXTRACT(YEAR FROM a.transaction_month) AS transaction_year,
  b.cust_id,
  a.transaction_month AS active_current_mth,
  a.Activity_seg,
  a.flag,
  a.pp_bt_cust_id,
  a.cust_acct_clsfn_key,
  a.sales_seg,
  a.market,
  a.sub_market,
    a.Region,
    a.cntry_code,
    a.prod_grp_l1,
    a.prod_grp_l2,
    a.prod_grp_l3,
    a.trade_flow,
    a.txn_prod_channel,
    a.new_x_sell_retention_flag,
    a.mercury_sector_name,
    a.mercury_industry_name,
    a.Managed_flag,
    a.account_type,
FROM
  `pypl-bods`.prd_globalbi_sales_t.merchant_base_final_table_v2 a
JOIN
   cohort_data b
ON
  a.cust_id = b.cust_id
GROUP BY
  1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23

HTML(value='')

job id =  8PwYeOgB4RHIY23FUwY1LQK4Kso
Time executed =  -63840676687.584  seconds


HTML(value='')

DownloadWidget(children=(HBox(children=(Dropdown(description='File Type:', index=1, options=('tde', 'csv', 'js…

cohort_year,transaction_year,cust_id,active_current_mth,Activity_seg,flag,pp_bt_cust_id,cust_acct_clsfn_key,sales_seg,market,sub_market,Region,cntry_code,prod_grp_l1,prod_grp_l2,prod_grp_l3,trade_flow,txn_prod_channel,new_x_sell_retention_flag,mercury_sector_name,mercury_industry_name,Managed_flag,account_type


### Segment KPI

In [7]:
create or replace table `pypl-bods`.prd_globalbi_sales_t.merchant_segment_intermediate_v1
as (
select cust_id,entity_id,mth_id from
(
select 
mth_id
,cust_id
,entity_id
from `pypl-edw`.pp_ent_anlytcs_views.fact_merch_prod_hier_mth 
where mth_id >= 1453  -- replace with Jan'21 till last month 1453 to 1487
and sales_seg in ('Global Accounts','A1','A2','B1','B2','C','D','E')  
and prod_hier_desc not in ('eBay MOR','eBay Non-MOR')
and cust_acct_clsfn_key in (2,8,-99,21) 
and (is_partner_y_n is null or is_partner_y_n <>'Y')
and cust_id<>'0'
group by 1,2,3
qualify row_number() over(partition by cust_id order by mth_id desc)=1)
group by 1,2,3);

HTML(value='')

job id =  Pwc9MFhXEYR8t32ZQoIDaoshTKX
Time executed =  -63840677393.668  seconds


HTML(value='')

DownloadWidget(children=(HBox(children=(Dropdown(description='File Type:', index=1, options=('tde', 'csv', 'js…

cust_id,entity_id,mth_id


In [ ]:
create or replace table `pypl-bods`.prd_globalbi_sales_t.merchant_segment_intermediate_v2
as (
    select 
a.mth_id,
parse_date('%Y-%m',b.mth_yyyy_mm_name) as transaction_month
,a1.cust_id
,a1.entity_id,
a.pp_bt_cust_id,
a.cust_acct_clsfn_key,
case when upper(trim(a.sales_seg)) in ('GLOBAL ACCOUNTS') then 'GA'
 When upper(trim(a.sales_seg)) in ('A1','A2','B1') then 'ES'
 When upper(trim(a.sales_seg)) in('B2') then 'MM'
 When upper(trim(a.sales_seg)) in ('C','D','E') then 'SB'
 Else '#'
 END as sales_seg,
a.market,
a.sub_market,
CASE WHEN a.sub_market in ('UK', 'CHINA') then a.sub_market
when a.cntry_code in ('US', 'CA', 'DE', 'AU', 'FR', 'IT','ES', 'BR','MX', 'JP') then a.cntry_code
when a.sub_market in ('DEATCH', 'ICE', 'NEU', 'SEU') and a.cntry_code not in ('US', 'CA', 'DE', 'AU', 'FR', 'IT','ES', 'BR','MX', 'JP', 'CN') then 'Rest of Europe'
else 'Rest of World' end as Region,  
a.cntry_code,   
CASE WHEN upper(trim(a.prod_hier_desc))='BRAINTREE' then 'Braintree' 
     WHEN upper(trim(a.prod_hier_l1)) IN ('BRANDED WALLET-DIRECT', 'BRANDED WALLET-MARKETPLACE','BRANDED WALLET-PARTNERS', 'EBAY') THEN 'Wallet'
     WHEN upper(trim(a.prod_hier_l1)) IN ('UNBRANDED CARD PROCESSING-DIRECT', 'UNBRANDED CARD PROCESSING-MARKETPLACE','UNBRANDED CARD PROCESSING-PARTNERS') THEN 'PP-Cards'     
     WHEN upper(trim(a.prod_hier_l1)) IN ('APM-DIRECT', 'APM-MARKETPLACE', 'APM-PARTNERS', 'PWV-DIRECT', 'PWV-MARKETPLACE', 'PWV-PARTNERS',
                                         'CREDIT-DIRECT', 'CREDIT-MARKETPLACE', 'CREDIT-PARTNERS') THEN 'Other-XO'
     ELSE 'Non-XO'
END prod_grp_l1,   
CASE WHEN upper(trim(a.prod_hier_desc))='BRAINTREE' then 'Braintree' -- Braintree 
     WHEN upper(trim(a.prod_hier_l3)) in ('EC - LATEST', 'EC - BETTER', 'EC - LEGACY', 'PUI - BETTER', 'PUI - LEGACY') and upper(trim(a.prod_hier_desc)) <> 'WPS NON RECURRING' then 'EC' --Wallet
     WHEN (upper(trim(a.prod_hier_l3)) in ('WPS - LEGACY') or upper(trim(a.prod_hier_desc)) = 'WPS NON RECURRING') and (upper(trim(a.prod_hier_l3)) <> 'INTEGRATED QR') then 'WPS'   --Wallet
     WHEN upper(trim(a.prod_hier_l1)) = 'EBAY' THEN 'eBay'  --Wallet
     WHEN upper(trim(a.prod_hier_l2)) in ('ACDC', 'BCDC', 'PRO DCC') THEN a.prod_hier_l2  -- PP Cards
     WHEN upper(trim(a.prod_hier_l1)) like 'UNBRANDED CARD PROCESSING' and upper(trim(a.prod_hier_l2)) not in ('ACDC', 'BCDC', 'PRO DCC', 'BRAINTREE') THEN 'PPCards-Oth' -- PP Cards
     WHEN upper(trim(a.prod_hier_l1)) IN ('APM-DIRECT', 'APM-MARKETPLACE', 'APM-PARTNERS','PWV-DIRECT', 'PWV-MARKETPLACE',
                                          'PWV-PARTNERS') THEN 'APM (INCL PWV)' -- Other XO
     WHEN upper(trim(a.prod_hier_l1)) IN ('CREDIT-DIRECT', 'CREDIT-MARKETPLACE', 'CREDIT-PARTNERS') THEN 'Credit' -- Other XO
     WHEN upper(trim(a.prod_hier_l2)) in ('INVOICING', 'ZETTLE', 'PAYOUTS') then a.prod_hier_l2
     WHEN upper(trim(a.prod_hier_l2)) in ('P2P-F&F', 'P2P-G&S') then 'P2P'
     ELSE 'Oth'
END AS prod_grp_l2,
CASE WHEN upper(trim(a.prod_hier_desc)) = 'BRAINTREE' then 'Braintree' -- Braintree
     WHEN upper(trim(a.prod_hier_l1)) = 'EBAY' THEN 'eBay' --Wallet
     WHEN upper(trim(a.prod_hier_desc)) = 'MS EC RECURRING' and upper(trim(a.prod_hier_l2)) not in ('APM' , 'PWV', 'PPC 2ND BTN', 'ACDC', 'BCDC') then 'EC Recurring'  --Wallet
     WHEN upper(trim(a.prod_hier_desc)) = 'MS EC Non Recurring' and upper(trim(a.prod_hier_l2)) not in ('APM' , 'PWV', 'PPC 2ND BTN', 'ACDC', 'BCDC') then 'EC Non Recurring' --Wallet
     WHEN upper(trim(a.prod_hier_desc)) in ('WPS RECURRING', 'WPS NON RECURRING')  and  upper(trim(a.prod_hier_l2)) not in ('APM' , 'PWV', 'PPC 2ND BTN', 'ACDC', 'BCDC', 'INTEGRATED QR')  then a.prod_hier_desc --Wallet
     WHEN upper(trim(a.prod_hier_l3)) in ('EC - LATEST', 'EC - BETTER', 'EC - LEGACY', 'PUI - BETTER', 'PUI - LEGACY') then 'EC Non Recurring' --Wallet
     WHEN upper(trim(a.prod_hier_l3)) in ('WPS - LEGACY') then 'WPS Non Recurring'  --Wallet
     WHEN upper(trim(a.prod_hier_l2)) in ('ACDC', 'BCDC', 'PRO DCC') THEN a.prod_hier_l2 -- PP Cards     
     WHEN upper(trim(a.prod_hier_l2)) = 'HSS UNBRANDED' then 'HSS' -- PP Cards 
     WHEN upper(trim(a.prod_hier_l2)) = 'PP PLUS - UNBRANDED' then 'PP Plus'  -- PP Cards 
     WHEN upper(trim(a.prod_hier_l2)) = 'PPFM UNBRANDED' then 'PPFM'  -- PP Cards 
     WHEN upper(trim(a.prod_hier_l1)) IN ('APM-DIRECT', 'APM-MARKETPLACE', 'APM-PARTNERS') THEN 'APM'  -- Other XO
     WHEN upper(trim(a.prod_hier_l1)) IN ('PWV-DIRECT', 'PWV-MARKETPLACE', 'PWV-PARTNERS') THEN 'PWV'  -- Other XO
     WHEN upper(trim(a.prod_hier_l1)) IN ('CREDIT-DIRECT', 'CREDIT-MARKETPLACE', 'CREDIT-PARTNERS') THEN 'Credit'  -- Other XO
     WHEN upper(trim(a.prod_hier_l2)) in ('INVOICING', 'ZETTLE', 'P2P-G&S', 'VIRTUAL TERMINAL', 'PAYPAL HERE') then 
         upper(trim(a.prod_hier_l2))
     WHEN upper(trim(a.prod_hier_l1)) IN ('P2P-F&F RECV', 'P2P-F&F SEND', 'MS OTHERS', 'PAYOUTS RECV', 'PAYOUTS SEND') 
         then upper(trim(a.prod_hier_l1))
     WHEN upper(trim(a.prod_hier_l2)) in ('INTEGRATED QR', 'STATIC QR') then 'QRC' 
     WHEN upper(trim(a.prod_hier_l2)) = 'TOKENIZATION & DISCOVER POS' THEN 'POS'
     ELSE 'MS OTHERS'
END AS prod_grp_l3,
case when a.trade_flow = 'NA' then 'BT'
else a.trade_flow
end as trade_flow,
a.txn_prod_channel,
a.new_x_sell_retention_flag,
-- a.eds_flag,
-- a.ews_flag,
-- a.ems_flag,
case 
when new_flag='Y' then 'New'
when eds_flag='Y' then 'EDS'
when ews_flag='Y' then 'EWS'
when ems_flag='Y' then 'EMS'
else 'Rest' end as Activity_seg,   
d.mercury_sector_name as mercury_sector_name,
d.mercury_industry_name as mercury_industry_name,
CASE 
WHEN upper(trim(m.cust_id)) is not null or upper(trim(a.prod_hier_desc))='EBAY MOR' then 'MANAGED PATH' 
WHEN upper(trim(a.txn_prod_channel)) in ('PARTNER','MARKETPLACE') or 
upper(trim(a.prod_hier_desc))='EBAY NON-MOR' then 'CONNECTED PATH'
else 'DIRECT' 
END as Managed_flag,
CASE
WHEN a.cust_acct_type_code = 0 THEN 'Personal'
WHEN a.cust_acct_type_code = 1 THEN 'Premier'
ELSE 'Business'
END AS account_type,
sum(rcv_txn_cnt)  txn_cnt,
sum(rcv_ntpv_pln_usd_amt ) ntpv,
sum(a.net_rev_pln_usd_amt) AS revenue,
sum(a.net_margin_pln_usd_amt) AS margin,
sum(a.net_cost_pln_usd_amt) AS cost,
sum(a.net_loss_pln_usd_amt) AS loss
from `pypl-edw`.pp_ent_anlytcs_views.fact_merch_prod_hier_mth a
--LEFT JOIN `pypl-edw`.pp_discovery_views.dim_customer AS attr 
--ON upper(rtrim(a.pp_bt_cust_id)) = upper(rtrim(attr.cust_id))
LEFT OUTER JOIN `pypl-edw`.pp_discovery_views.dim_calendar_month AS b 
ON trim(CAST(a.mth_id as STRING)) = trim(CAST(b.mth_id as STRING))
LEFT JOIN (select cust_id,mercury_sector_name,mercury_industry_name from `pypl-edw`.pp_scratch.mercury_industry_details
where dt=(select max(dt) from pypl-edw.pp_scratch.mercury_industry_details)
group by 1,2,3) d on a.cust_id = d.cust_id  
LEFT JOIN  pypl-edw.pp_peak_views.dim_mam_merch_ssot m 
on upper(rtrim(a.pp_bt_cust_id)) = upper(rtrim(m.cust_id))
LEFT JOIN `pypl-bods`.prd_globalbi_sales_t.merchant_segment_intermediate_v1 a1
on upper(rtrim(a.cust_id)) = upper(trim(a1.cust_id))
where a.mth_id >= 1453 -- replace with Jan'21 till last month 1453 to 1487
   and upper(trim(a.sales_seg)) in ('A1','A2','B1','B2','C','D','E','GLOBAL ACCOUNTS')  
    and a.cust_acct_clsfn_key in (2,8,-99,21)
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22);

HTML(value='')

job id =  sVO1vHe2TadoUMvvNEx3syAt748
Time executed =  -63840677436.564  seconds


In [11]:
select distinct cust_acct_clsfn_key from `pypl-bods`.prd_globalbi_sales_t.merchant_segment_intermediate_v2 
group by 1

HTML(value='')

job id =  CMfkncuyJsRNxbaTDVgNM81OGMk
Time executed =  -63840573388.959  seconds
.

HTML(value='')

DownloadWidget(children=(HBox(children=(Dropdown(description='File Type:', index=1, options=('tde', 'csv', 'js…

cust_acct_clsfn_key
21
8
2
-99
